In [63]:
import warp as wp
import numpy as np
import trimesh

from warp.sim.render import *

from warp.sim.collide import TriMeshCollisionDetector

In [64]:
device = wp.get_device('cuda')
# if you don't have a CUDA-compatible GPU try switching to CPU
# device = wp.get_device('cpu')

In [65]:
mesh = trimesh.load('A00000500.ply')


In [66]:
builder  = wp.sim.ModelBuilder()

vertices = [wp.vec3(mesh.vertices[i,:]) for i in range(mesh.vertices.shape[0])]
builder.add_cloth_mesh(
            pos=wp.vec3(0.0, 0.0, 0.0),
            rot=wp.quat_identity(),
            scale=1.0,
            vertices=vertices,
            indices=mesh.faces.reshape(-1),
            vel=wp.vec3(0.0, 0.0, 0.0),
            density=0.02,
            tri_ke=1.0e5,
            tri_ka=1.0e5,
            tri_kd=2.0e-6,
            edge_ke=10,
)
builder.color()
model = builder.finalize()

Warp UserWarning: The graph is not optimizable anymore, terminated with a max/min ratio: 2.0 without reaching the target ratio: 1.1


In [67]:
# to access ForceElementAdjacencyInfo, you need to construct a VBDIntegrator (you dont need to understand what it is)
vbd_integrator = wp.sim.VBDIntegrator(model)

In [68]:
colision_detector = TriMeshCollisionDetector(model)

In [69]:
colision_detector.vertex_triangle_collision_detection(5.0)
colision_detector.edge_edge_collision_detection(5.0)

In [70]:
# d^v_{min}
print(colision_detector.vertex_colliding_triangles_min_dist)
# d^E_{min}
print(colision_detector.edge_colliding_edges_min_dist)
# d^T_{min}
print(colision_detector.triangle_colliding_vertices_min_dist)

[2.0408168 1.3784707 1.7306733 ... 1.7102327 1.3814771 2.0408168]
[1.3784707 1.8084555 1.514926  ... 1.3814771 2.012112  1.515333 ]
[1.4946659 1.3784707 1.6526986 ... 1.6386598 1.3814771 1.5250734]


In [72]:
from warp.sim.integrator_vbd import get_vertex_num_adjacent_edges, get_vertex_adjacent_edge_id_order, get_vertex_num_adjacent_faces, get_vertex_adjacent_face_id_order, ForceElementAdjacencyInfo
# how to iterate over neighbor elements
@wp.kernel
def iterate_vertex_neighbor_primitives(
    adjacency: ForceElementAdjacencyInfo
):
    particle_idx = wp.tid()

    # iterating over neighbor faces
    num_adj_faces = get_vertex_num_adjacent_faces(adjacency, particle_idx)
    for face_counter in range(num_adj_faces):
        adj_face_idx, vertex_order = get_vertex_adjacent_face_id_order(adjacency, particle_idx, face_counter)
    # iterating over neighbor edges
    num_adj_edges = get_vertex_num_adjacent_edges(adjacency, particle_idx)
    for edge_counter in range(num_adj_edges):
        edge_idx, v_order = get_vertex_adjacent_edge_id_order(adjacency, particle_idx, edge_counter)

wp.launch(
    iterate_vertex_neighbor_primitives,
    dim=model.particle_count,
    inputs=[vbd_integrator.adjacency],
    device=device
)

In [73]:
# your turn: you need to:
# Implement conservative bounds computation using the instructions provided above
# it must be implemented using @warp.kernel to maximize efficiency

In [74]:

@wp.kernel
def compute_conservative_bounds(
    adjacency: wp.sim.integrator_vbd.ForceElementAdjacencyInfo,
    vertex_triangle_min_dist: wp.array(dtype=wp.float32),  # d_min,v 
    edge_edge_min_dist: wp.array(dtype=wp.float32),        # d_min,e 
    triangle_vertex_min_dist: wp.array(dtype=wp.float32),  # d_min,t 
    conservative_bounds: wp.array(dtype=wp.float32),       # output: b_v for each vertex
    gamma_v: wp.float32 = 0.4  #  (0 < γ_v < 0.5)
):
    particle_idx = wp.tid()
    
    # vertex to triangle minimal distance)
    d_min_v = vertex_triangle_min_dist[particle_idx]
    
    # minimal edge distance for this vertex
    d_E_min_v = wp.float32(1e10)  # Initialize to large value
    num_adj_edges = wp.sim.integrator_vbd.get_vertex_num_adjacent_edges(adjacency, particle_idx)
    for edge_counter in range(num_adj_edges):
        edge_idx, v_order = wp.sim.integrator_vbd.get_vertex_adjacent_edge_id_order(adjacency, particle_idx, edge_counter)
        edge_min_dist = edge_edge_min_dist[edge_idx]
        d_E_min_v = wp.min(d_E_min_v, edge_min_dist)
    
    # minimal triangle distance for this vertex
    d_T_min_v = wp.float32(1e10)  # Initialize to large value
    num_adj_faces = wp.sim.integrator_vbd.get_vertex_num_adjacent_faces(adjacency, particle_idx)
    for face_counter in range(num_adj_faces):
        adj_face_idx, vertex_order = wp.sim.integrator_vbd.get_vertex_adjacent_face_id_order(adjacency, particle_idx, face_counter)
        triangle_min_dist = triangle_vertex_min_dist[adj_face_idx]
        d_T_min_v = wp.min(d_T_min_v, triangle_min_dist)
    
    # Compute conservative bound: b_v = γ_v * min(d_min,v, d^E_min,v, d^T_min,v)
    min_distance = wp.min(wp.min(d_min_v, d_E_min_v), d_T_min_v)
    conservative_bounds[particle_idx] = gamma_v * min_distance

conservative_bounds = wp.zeros(model.particle_count, dtype=wp.float32, device=device)

# Launch the conservative bounds kernel
wp.launch(
    compute_conservative_bounds,
    dim=model.particle_count,
    inputs=[
        vbd_integrator.adjacency,
        colision_detector.vertex_colliding_triangles_min_dist,
        colision_detector.edge_colliding_edges_min_dist, 
        colision_detector.triangle_colliding_vertices_min_dist,
        conservative_bounds,
        0.4  # gamma_v parameter
    ],
    device=device
)

print("Conservative bounds:", conservative_bounds)

Conservative bounds computed: [0.55138826 0.55138826 0.55138826 ... 0.55259085 0.55259085 0.55259085]
